In [1]:
import numpy as np
import pandas as pd

import os
import pickle

In [2]:
patch_path = '/project/GutIntelligenceLab/ss4yd/gtex_data/process_path_level1/'
dirs = os.listdir(patch_path)
dir_patch_dict = {}
for pid in dirs:
    patches = [os.path.join(patch_path, pid, x) for x in os.listdir(os.path.join(patch_path, pid))]
    dir_patch_dict[pid] = patches
    
patches = [x for xs in dir_patch_dict.values() for x in xs]

In [3]:
df = pd.DataFrame(patches, columns=['patch_paths'])
df['pid'] = df['patch_paths'].apply(lambda x: x.split('/')[-2])
df

,patch_paths,pid
0,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R45C-1426
1,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R45C-1426
2,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R45C-1426
3,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R45C-1426
4,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R45C-1426
...,...,...
715178,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-1I1GV-3026
715179,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-1LC47-0426
715180,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-1LC47-0426
715181,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-1LC47-0426


In [4]:
counts = df.pid.value_counts()
counts

GTEX-1IKJJ-3026    1458
GTEX-147GR-1426    1125
GTEX-13CIG-1426     858
GTEX-1IDJV-0826     747
GTEX-15UF7-0426     747
                   ... 
GTEX-ZG7Y-2426        1
GTEX-14ICL-2026       1
GTEX-1GZ4I-0226       1
GTEX-1LSNL-2826       1
GTEX-1F75W-0926       1
Name: pid, Length: 5390, dtype: int64

In [5]:
counts[counts>50]

GTEX-1IKJJ-3026    1458
GTEX-147GR-1426    1125
GTEX-13CIG-1426     858
GTEX-1IDJV-0826     747
GTEX-15UF7-0426     747
                   ... 
GTEX-PVOW-1026       51
GTEX-144FL-0726      51
GTEX-W5X1-1626       51
GTEX-14E7W-2526      51
GTEX-WHPG-0426       51
Name: pid, Length: 3912, dtype: int64

In [6]:
filter_index = list(counts[counts>50].index)
fdf = df[df.pid.isin(filter_index)]
fdf

,patch_paths,pid
0,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R45C-1426
1,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R45C-1426
2,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R45C-1426
3,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R45C-1426
4,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R45C-1426
...,...,...
715174,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-1I1GV-3026
715175,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-1I1GV-3026
715176,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-1I1GV-3026
715177,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-1I1GV-3026


In [8]:
# unique_pid = pd.unique(fdf.pid)
# unique_pid.split((0.75,0.15,0.10))

In [7]:
with open('../paths.pickle', 'rb') as file:
    paths = pickle.load(file)

In [8]:
reps = np.load('/project/GutIntelligenceLab/ss4yd/gtex_data/all_reps.npy')

reps.shape

len(paths)

repsdf = pd.DataFrame(reps)

repsdf['paths'] = paths

# train, validate, test = np.split(repsdf.sample(frac=1, random_state=42), [int(.75*len(repsdf)), int(.9*len(repsdf))])

# train['dtype']='train'
# validate['dtype']='val'
# test['dtype']='test'

# repsdf = pd.concat([train, validate, test])
repsdf['pid'] = repsdf['paths'].apply(lambda x: x.split('/')[-2])

split = pd.read_csv('../csv/train_test_val_split.csv')

repsdf = repsdf.merge(split, on='pid')

print("Len before filter: {}".format(len(repsdf)))
repsdf = repsdf[repsdf.pid.isin(filter_index)]
print("Len after filter: {}".format(len(repsdf)))

Len before filter: 715136
Len after filter: 679076


In [9]:
repsdf.to_csv('/project/GutIntelligenceLab/ss4yd/gtex_data/reps_paths.csv', index=False)